<a href="https://colab.research.google.com/github/laxmipathiraju1214/EVA4/blob/master/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,8, 3, bias= False)
        self.conv2 = nn.Conv2d(8,16, 3, bias= False)
        self.conv3 = nn.Conv2d(16,32, 3, bias= False)
        self.pool = nn.MaxPool2d(2,2)
        self.conv11 = nn.Conv2d(32,16,1, bias=False)
        #self.conv112 = nn.Conv2d(32,16,1, bias=False)
        self.conv4 = nn.Conv2d(16,16,3,bias=False)
        self.conv5 = nn.Conv2d(16,22,3,bias=False)
        self.conv6 = nn.Conv2d(22,10,3,bias=False)
        self.gap = nn.AvgPool2d(5)
        self.bn32 = nn.BatchNorm2d(32)
        self.bn16 = nn.BatchNorm2d(16)
        self.bn8 = nn.BatchNorm2d(8)
        self.bn22 = nn.BatchNorm2d(22)
        self.dropout1 = nn.Dropout2d(0.08)
        self.dropout2 = nn.Dropout2d(0.08)

        

    def forward(self, x):
       x = self.pool(self.dropout1(self.bn32(F.relu(self.conv3(self.dropout1(self.bn16(F.relu(self.conv2(self.dropout1(self.bn8(F.relu(self.conv1(x)))))))))))))
       x = self.dropout1(self.bn16(F.relu(self.conv11(x))))
       x = self.dropout2(self.bn16(F.relu(self.conv4(x))))
       #x = self.dropout2(self.bn16(F.relu(self.conv112(x))))
       x = self.gap(self.conv6(self.dropout2(self.bn22(F.relu(self.conv5(x))))))
       x = x.view(-1,10)
       return F.log_softmax(x)

In [195]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
       BatchNorm2d-2            [-1, 8, 26, 26]              16
         Dropout2d-3            [-1, 8, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           1,152
       BatchNorm2d-5           [-1, 16, 24, 24]              32
         Dropout2d-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 32, 22, 22]           4,608
       BatchNorm2d-8           [-1, 32, 22, 22]              64
         Dropout2d-9           [-1, 32, 22, 22]               0
        MaxPool2d-10           [-1, 32, 11, 11]               0
           Conv2d-11           [-1, 16, 11, 11]             512
      BatchNorm2d-12           [-1, 16, 11, 11]              32
        Dropout2d-13           [-1, 16, 11, 11]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.02f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [198]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0712, Accuracy: 9810/10000 (98.10%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0623, Accuracy: 9817/10000 (98.17%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0234, Accuracy: 9927/10000 (99.27%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0236, Accuracy: 9927/10000 (99.27%)



loss=0.023710766807198524 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 42.34it/s]



Test set: Average loss: 0.0230, Accuracy: 9932/10000 (99.32%)

